# Example of how to use a model with `heiplanet-models`

In [ ]:
import model_backend as mb
from pathlib import Path
import xarray as xr
import rioxarray as rioxr
import numpy as np
from copy import deepcopy
from datetime import datetime
import json

## Set up the variables to run the model 

data is assumed to live in a directory: 

data/in

relative to the base directory of the package

In [ ]:
path = Path.cwd().parent / "data" / "in"
path

In [ ]:
data_file = "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc"

In [ ]:
ro_file = "R0_pip_stats.csv"

## make output directory

In [ ]:
outpath = Path.cwd().parent / "data" / "out" / datetime.today().strftime("%Y-%m-%d")
outpath.mkdir(parents=True, exist_ok=True)

## define the model config 

do the following: 
- read the default config 


In [ ]:
default_config = mb.Jmodel.read_default_config()

In [ ]:
default_config

update the config with anything you might want

In [ ]:
config = deepcopy(default_config)
config["graph"]["setup_modeldata"]["kwargs"]["input"] = str(path / data_file)
config["graph"]["setup_modeldata"]["kwargs"]["r0_path"] = str(path / ro_file)
config["graph"]["setup_modeldata"]["kwargs"]["output"] = str(
    outpath / f"output_jmodel_europe.nc"
)

with open(outpath / "config_europe.json", "w") as f:
    json.dump(config, f)

In [ ]:
computation = mb.computation_graph.ComputationGraph(config)

In [ ]:
computation.visualize(outpath / "computation_jmodel.png")

## Run model -> load data -> apply model transformations -> save back to netcdf

In [ ]:
computation.execute()

In [ ]:
data = xr.open_dataset(path / data_file, chunks=None)
data

access individual nodes via the task graph

In [ ]:
input = computation.task_graph["read_input_data"].compute()

In [ ]:
input["t2m"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
input["t2m"].sel(valid_time="2024-08-01").plot(cmap="viridis")

## Have a look at produced data

In [ ]:
data = xr.open_dataset(outpath / "output_jmodel_europe.nc", engine="rasterio")

In [ ]:
data

In [ ]:
data["R0"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
data["R0"].sel(valid_time="2024-08-01").plot(cmap="viridis")

## Run model with parallel run mode on a global scale

In [ ]:
output = outpath / "output_JModel_global.nc"
run_mode = "parallelized"
grid_data_baseurl = None
nuts_level = None
resolution = None
year = None
global_config = deepcopy(default_config)
global_config["graph"]["setup_modeldata"]["kwargs"]["input"] = str(path / data_file)
global_config["graph"]["setup_modeldata"]["kwargs"]["r0_path"] = str(path / ro_file)
global_config["graph"]["setup_modeldata"]["kwargs"]["output"] = str(output)
global_config["graph"]["setup_modeldata"]["kwargs"]["run_mode"] = run_mode
global_config["graph"]["setup_modeldata"]["kwargs"]["grid_data_baseurl"] = (
    grid_data_baseurl
)
global_config["graph"]["setup_modeldata"]["kwargs"]["nuts_level"] = nuts_level
global_config["graph"]["setup_modeldata"]["kwargs"]["resolution"] = resolution
global_config["graph"]["setup_modeldata"]["kwargs"]["year"] = year

In [ ]:
global_config

In [ ]:
with open(outpath / "config_global.json", "w") as f:
    json.dump(config, f)

In [ ]:
computation_global = mb.computation_graph.ComputationGraph(global_config)

In [ ]:
computation_global.execute()

In [ ]:
input = computation_global.task_graph["read_input_data"].compute()

In [ ]:
input["t2m"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
data = xr.open_dataset(output, engine="rasterio", chunks=None).compute()

In [ ]:
data["R0"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
data["R0"].sel(valid_time="2024-08-01").plot(cmap="viridis")

# Monthly data

## Build new model instance 

In [ ]:
data_file = (
    "era5_data_2024_01_02_03_2t_tp_monthly_celsius_mm_resampled_05degree_trim.nc"
)

In [ ]:
output = outpath / "output_JModel_monthly.nc"
monthly_config = deepcopy(default_config)
monthly_config["graph"]["setup_modeldata"]["kwargs"]["input"] = str(path / data_file)
monthly_config["graph"]["setup_modeldata"]["kwargs"]["output"] = str(output)
monthly_config["graph"]["setup_modeldata"]["kwargs"]["r0_path"] = str(path / ro_file)
monthly_config["graph"]["setup_modeldata"]["kwargs"]["grid_data_baseurl"] = (
    "https://gisco-services.ec.europa.eu/distribution/v2/nuts"
)

In [ ]:
with open(outpath / "config_monthly.json", "w") as f:
    json.dump(config, f)

In [ ]:
computation = mb.computation_graph.ComputationGraph(config=monthly_config)

In [ ]:
computation.execute()

In [ ]:
data = computation.task_graph["read_input_data"].compute()
data

In [ ]:
data["t2m"].mean(dim="time").plot(cmap="viridis")

In [ ]:
data["t2m"].sel(time="2024-01-01").plot(cmap="viridis")

In [ ]:
data = xr.open_dataset(
    monthly_config["graph"]["setup_modeldata"]["kwargs"]["output"],
    engine="rasterio",
    chunks=None,
).compute()

In [ ]:
data["R0"].mean(dim="time").plot(cmap="viridis")

since it's always summer in French Guiana by comparison, we have high transmissivity there, but literally nothing in europe where it's cold. 

In [ ]:
data["R0"].sel(time="2024-01-01").plot(cmap="viridis")